In [ ]:
import pandas as pd
import numpy as np
import os
import glob

pd.options.display.max_columns = None
pd.options.display.max_rows = None

inpath = '/MFMDatasets/MFM_bopf/src/multi_predict/'
#inpath = '/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/'
outpath = '/MFMDatasets/MFM_bopf/src/multi_predict/results/'
out_flag = True
#out_flag = False

Cutoff_stat = "Fb2_2"
SORT_STAT = "Fb2_2"
#Cutoff_stat = "MCC"
#SORT_STAT = "MCC"
#Cutoff_stat = "PR_AUC"
#SORT_STAT = "PR_AUC"
#Cutoff_stat = "ROC_AUC"
#SORT_STAT = "ROC_AUC"

# Cut off value for given stat.: NOTE: latter parts of notebook won't work due to missing indexes
CUTOFF = 0.0


#alg_cols = ['CLF_time(min)','target','under_alg']
alg_cols = ['CLF_time(min)','target','period','feats','under_alg']
#alg_cols = ['CLF_time(min)','target','under_alg','samp_strat']
#stat_cols = ['TP','FN','FP','TN','precision_2','recall_2','Spec_2',
stat_cols = ['TP','FN','FP','TN','NTP','NFN','NFP','NTN','precision_2','recall_2','Spec_2',
            'ROC_AUC','PR_AUC','MCC','Fb1_2','Fb2_2','Fb05_2',
            'Gmean_2','Gmean_ma','Max Gmean','Max Thresh'
            ]

#period = "-PI"
period = "-Pre"

#alg = "LR"
#spec_cols = ['p_C','p_solver']

#alg = "NB"
#spec_cols = ['p_var_smoothing']

#alg = "SVC"
#spec_cols = ['p_C','p_kernel','p_degree']

#alg = "MLP"
#spec_cols = ['p_alpha','p_hidden_layer_sizes']

alg = "RF"
spec_cols = ['p_n_estimators','p_max_depth','p_min_samples_leaf','p_min_samples_split']

#alg = "GB"
#spec_cols = ['p_n_estimators','p_max_depth','p_subsample','p_learning_rate']

alg_cols.extend(spec_cols)
alg_cols.extend(stat_cols)

os.chdir(inpath + alg + '/output')
#os.chdir(inpath + alg + '/2021_02_Cramer/output')
#os.chdir(inpath + alg + '/Oct2020/output')
#os.chdir(inpath + alg + '/Nov2020/output')

#dataset = "transfus_yes"
#dataset = "Hysterectomy"
dataset = "*"
#dataset = "trans*"
ext = '.csv'
glob_string = dataset + period + "-*" + alg + "-*"
files = glob.glob(glob_string + ext)
#files = glob.glob("*-" + alg + "-*" + ext)
#files = glob.glob("transfus_yes*-" + alg + "-*" + ext)

results_df = pd.DataFrame()
#os.chdir('/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/MLP/output')
#os.chdir('/slurm_storage/mbopf/projects/MFM_bopf/src/multi_predict/SVC/output')
#os.chdir('/MFMDatasets/MFM_bopf/src/multi_predict/NB/output')
#files = glob.glob("*-NB-*.csv")
#files = glob.glob("*-MLP-*.csv")
#files = glob.glob("*-SVC-*.csv")
#files = glob.glob("transfus_yes*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*]_0.*_500*.csv")
#files = glob.glob("transfus_hyster*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*.csv")
#files = glob.glob("transfus_yes*-MLP-*.csv")
len(files)

In [ ]:
def remove_dups(file):
    base = glob.escape(file[:-4])
    print(f'base = {base}')
    dup_files = glob.glob(base + '*')
    print(dup_files)
    for dup_file in dup_files:
        print(f'Removing duplicate: {dup_file}')
        os.remove(dup_file)

In [ ]:
files = sorted(files, reverse=True)
for i in range(len(files)-1):
    if files[i][:-20] == files[i+1][:-20]:
        print(f'DUP:{files[i]}')
        #remove_dups(files[i])

files = glob.glob(glob_string + ext)
files = sorted(files, reverse=True)
print(f'len(files) = {len(files)}')

In [ ]:
prev_f = ""
for idx, f in enumerate(files):
    print(f'idx={idx}; file={f}')
    if prev_f[:-20] == f[:-20]:   #if the filenames only differ by timestamp
        print(f'DUP:{prev_f}')
        continue   #skip "duplicate"
        
    if results_df.empty:
        results_df = pd.read_csv(f, header=None, index_col=0, names=[idx])
    else:
        this_df = pd.read_csv(f, header=None, index_col=0, names=[idx])
        results_df = results_df.merge(this_df, left_index=True, right_index=True, how="right")
    prev_f = f
results_df

In [ ]:
results_T = results_df.T
#results_T.sort_values(by=['ROC_AUC'], ascending=False, inplace=True)
#results_T.sort_values(by=['PR_AUC'], ascending=False, inplace=True)
#results_T.sort_values(by=['MCC'], ascending=False, inplace=True)
results_T[SORT_STAT] = pd.to_numeric(results_T[SORT_STAT])  # convert to float for sorting
results_T.sort_values(by=[SORT_STAT], ascending=False, inplace=True)
results_T['TP'] = results_T['TP'].astype(int)
results_T['FN'] = results_T['FN'].astype(int)
results_T['FP'] = results_T['FP'].astype(int)
results_T['TN'] = results_T['TN'].astype(int)
denom = results_T['TP'] + results_T['FN'] + results_T['FP'] + results_T['TN']
results_T.loc[:,'NTP'] = np.around(1000 * results_T['TP'] / denom, 0).astype(int)
results_T.loc[:,'NFN'] = np.around(1000 * results_T['FN'] / denom, 0).astype(int)
results_T.loc[:,'NFP'] = np.around(1000 * results_T['FP'] / denom, 0).astype(int)
results_T.loc[:,'NTN'] = np.around(1000 * results_T['TN'] / denom, 0).astype(int)

small = results_T[alg_cols]
small

In [ ]:
if out_flag:
    if dataset == '*':
        #small.to_csv(outpath + alg + '-ALL-' + SORT_STAT + '-Results.csv')
        small.to_csv(outpath + alg + '-' + SORT_STAT + '-Results.csv', float_format='%0.4f')
    else:
        small.to_csv(outpath + alg + "-" + dataset + '-' + SORT_STAT + '-Results.csv', float_format='%0.4f')

In [ ]:
# already in the correct directory from above: os.chdir(inpath + alg + '/output')
ext = '.out'
#files = glob.glob("*-" + alg + "-*" + ext)
#files = glob.glob("Hyster" + glob_prefix + ext)
files = glob.glob(glob_string + ext)
print(f'len(files) = {len(files)}')


In [ ]:
files = sorted(files, reverse=True)
for i in range(len(files)-1):
    #print(f'i={i}; file={files[i]}')
    if files[i][:-20] == files[i+1][:-20]:
        print(f'{files[i]}')
print(f'len(files) = {len(files)}')

In [ ]:
from tokenize import tokenize

high_coeff_files = set()
pd.options.display.float_format = '{:,.4f}'.format
prev_f = ""
coeffs_df = pd.DataFrame()
coeffs_rank_df = pd.DataFrame()
for idx, f in enumerate(files):
    print(f'idx={idx}; file={f}')
    if prev_f[:-20] == f[:-20]:   #if the filenames only differ by timestamp
        print(f'DUP:{prev_f}')
        continue   #skip "duplicate"
 
    with open(f, 'r') as file:
        found_coef = False
        cutoff = False
        coef_series = pd.Series()
        ind_vars = []
        coef_vals = []
        for line in file:
            # Cutoff stat
            if Cutoff_stat in line:
                cut_stat = line.split()[2]   
                if float(cut_stat) < CUTOFF:
                    cutoff = True
                    print(f'cut_stat = {cut_stat}; idx={idx}')
                    break    # Stop processing this file; poor performance
            
            if "coeffs =" in line:
                found_coef = True
                #print(f"found_coef: idx={idx}; file={f}")
                continue
            
            if found_coef:
                var_coef = line.split()
                if var_coef[0] != 'dtype:':
                    ind_vars.append(var_coef[0])
                    coef_vals.append(float(var_coef[1]))
                    if float(var_coef[1]) > 100:
                        high_coeff_files.add(f)
                
        if not cutoff and found_coef:
            if coeffs_df.empty:
                coeffs_df = pd.DataFrame(data=coef_vals, index=ind_vars, columns=[idx])
                coeffs_rank_df = pd.DataFrame(data=coeffs_df[idx].rank(ascending=False).astype(int), columns=[idx])
            else:
                this_df = pd.DataFrame(data=coef_vals, index=ind_vars, columns=[idx])
                this_rank_df = pd.DataFrame(data=this_df[idx].rank(ascending=False).astype(int), columns=[idx])
                coeffs_df = coeffs_df.merge(this_df, left_index=True, right_index=True, how="left")
                coeffs_rank_df = coeffs_rank_df.merge(this_rank_df, left_index=True, right_index=True, how="left")

    prev_f = f
    
coeffs_df

In [ ]:
#if out_flag:
#    if dataset == '*':
#        coeffs_df.to_csv(outpath + alg + '-ALL-coeffs.csv')
#    elif dataset == 'trans*':
#        coeffs_df.to_csv(outpath + alg + '-TRANS-coeffs.csv')
#    else:
#        coeffs_df.to_csv(outpath + alg + "-" + dataset + '-coeffs.csv')

In [ ]:
coeffs_rank_df['mean_rank'] = coeffs_rank_df.apply(np.mean, axis=1).rank().astype(int)
coeffs_rank_df = coeffs_rank_df.astype('Int64')
coeffs_rank_df.sort_values('mean_rank', inplace=True)
print(f'Count: {coeffs_rank_df.shape}')
len(coeffs_rank_df)
coeffs_rank_df

In [ ]:
#if out_flag:
#    if dataset == '*':
#        coeffs_rank_df.to_csv(outpath + alg + '-' + SORT_STAT + '-coef_rank.csv')
#    elif dataset == 'trans*':
#        coeffs_rank_df.to_csv(outpath + alg + '-TRANS-coef_rank.csv')
#    else:
#        coeffs_rank_df.to_csv(outpath + alg + "-" + dataset + '-coef_rank.csv')

In [ ]:
coeffs_rank_inv = coeffs_rank_df.T
combo = pd.concat([small, coeffs_rank_inv], axis=1, join='inner')
combo

In [ ]:
if out_flag:
    if dataset == '*':
        combo.to_csv(outpath + alg + '-' + SORT_STAT + '-combo.csv')
    else:
        combo.to_csv(outpath + alg + "-" + dataset + '-combo.csv')

In [ ]:
coeffs_df_inv = coeffs_df.T
#combo2 = pd.concat([combo, coeffs_df_inv['abruptio9']], axis=1, join='inner')
combo2 = pd.concat([combo, coeffs_df_inv], axis=1, join='inner')
combo2
#combo2.sort_values('abruptio9', inplace=True) # Can't do this because there are two "abruptio9" cols

In [ ]:
if out_flag:
    if dataset == '*':
        combo2.to_csv(outpath + alg + '-' + SORT_STAT + '-combo2.csv')
    else:
        combo2.to_csv(outpath + alg + "-" + dataset + '-combo2.csv')

In [ ]:
top_index = combo.iloc[0].name
print(f'top_index = {top_index}')
#top_index = 50
files[top_index]

In [ ]:
small.loc[top_index]

In [ ]:
base = glob.escape(files[top_index][:-20])
print(base)
#base = glob.escape(base)
#print(base)

In [ ]:
#path = inpath + alg + '/output/'
ft_glob = base + "*" + '_fpr_tpr.dat'
pr_glob = base + "*" + '_pr.dat'
pred_glob = base + "*" + '_pred.dat'
prob_glob = base + "*" + '_probs.dat'
print(glob.glob(ft_glob))
print(glob.glob(pr_glob))
print(glob.glob(pred_glob))
print(glob.glob(prob_glob))

In [ ]:
fpr, tpr, thresholds = np.loadtxt(glob.glob(ft_glob)[0], delimiter=',')
precision, recall, pr_thresh = np.loadtxt(glob.glob(pr_glob)[0], delimiter=',')
y_test, y_pred = np.loadtxt(glob.glob(pred_glob)[0], dtype=int, delimiter=',')
probs = np.loadtxt(glob.glob(prob_glob)[0], delimiter=',')
prob1 = probs[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, matthews_corrcoef, auc, roc_auc_score, precision_score
from imblearn.metrics import geometric_mean_score, sensitivity_specificity_support
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
roc_auc_s_prob = roc_auc_score(y_test, prob1)
print(f'ROC_AUC = {roc_auc_s_prob}')
pr_auc = auc(recall, precision)
print(f' PR_AUC = {pr_auc}')
#prec = precision_score(y_test, y_pred, pos_label=2)
prec = precision_score(y_test, y_pred)
print(f'   PREC = {prec}')
mcc = matthews_corrcoef(y_test, y_pred)
print(f'    MCC = {mcc}')
# Note: pos_label does not appear to have any effect
gmean_none = geometric_mean_score(y_test, y_pred, average=None)
print(f'gmean_none    = {gmean_none}')
gmean_macro = geometric_mean_score(y_test, y_pred, average='macro')
print(f'gmean_macro   = {gmean_macro}')
sens, spec, ss_supp = sensitivity_specificity_support(y_test, y_pred, average=None)
print(f'sens   = {sens}')
print(f'spec   = {spec}')

In [ ]:
print(f'y_test.shape = {y_test.shape}')
print(f'prob1.shape = {prob1.shape}')

In [ ]:
# roc_auc vs roc_auc_score
from sklearn.metrics import roc_auc_score, auc
roc_auc_s_prob = roc_auc_score(y_test, prob1)
roc_auc = auc(fpr, tpr)
print(f'roc_auc_s_prob = {roc_auc_s_prob}')
print(f'roc_auc(auc) = {roc_auc}')

In [ ]:
from sklearn.metrics import auc
import matplotlib.pyplot as plt
roc_auc = auc(fpr, tpr)
plt.title('ROC Curve')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('TPRate')
plt.xlabel('FPRate')
plt.show()

In [ ]:
# calculate the no skill line as the proportion of the positive class
no_skill = len(y_test[y_test==2]) / len(y_test)
# plot the no skill precision-recall curve
plt.plot([0, 1], [no_skill, no_skill], linestyle= '--', label= f'No Skill: {no_skill:.3f}')

pr_auc = auc(recall, precision)
plt.title('PR Curve')
plt.plot(recall, precision, 'g', label = 'AUC = %0.3f' % pr_auc)
plt.legend(loc = 'upper right')
plt.ylabel('Precision')
plt.xlabel('Recall')
plt.show()

In [ ]:
from sklearn.metrics import average_precision_score
print(f'PR_AUC = {pr_auc}')
print(f'AP     = {average_precision_score(y_test, prob1)}')

# Threshold Moving

In [ ]:
prob1

In [ ]:
plt.title('Probabilities')
plt.hist(prob1, bins=50)
plt.ylabel('Counts')
plt.xlabel('Minority probability')
plt.show()

In [ ]:
gmeans = np.sqrt(tpr * (1-fpr))
print(gmeans.shape)
plt.plot(gmeans)
plt.show()

In [ ]:
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print(f'max_ix = {ix}')
print( ' Best Threshold=%f, G-mean=%.3f ' % (thresholds[ix], gmeans[ix]))

In [ ]:
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, 'r', label='Logistic')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.show()

In [ ]:
print(f'ix = {ix}; fpr[ix] = {fpr[ix]}; tpr[ix]={tpr[ix]}')

In [ ]:
fscore = (2 * precision * recall) / (precision + recall)
ix = np.argmax(fscore)
print( ' Best Threshold=%f, F-measure=%.3f ' % (pr_thresh[ix], fscore[ix]))

In [ ]:
#no_skill = len(y_test[y_test==1]) / len(y_test)
#plt.plot([0,1], [no_skill,no_skill], linestyle= '--' , label= 'No Skill' )
plt.plot(recall, precision, label= 'PR curve' )
plt.scatter(recall[ix], precision[ix], marker= 'o' , color= 'black' , label= 'Best' )
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
# show the plot
plt.show()

In [ ]:
print(confusion_matrix(y_test, y_pred))
print(confusion_matrix(y_test, y_pred, normalize='true'))
print(confusion_matrix(y_test, y_pred, normalize='pred'))
print(confusion_matrix(y_test, y_pred, normalize='all'))

# Compare Cramer to "top index" coefficients

In [ ]:
print(f'top_index = {top_index}')
top_coeffs = pd.DataFrame(coeffs_df[top_index])
#top_coeffs = pd.DataFrame(coeffs_df[top_index], columns=['coeff'])
top_coeffs.rename(columns={top_index:'coeff'}, inplace=True)
top_coeffs.index.name = 'Variable'
top_coeffs.sort_values(by='coeff', ascending=False, inplace=True)
top_coeffs

In [ ]:
top_coeffs['coeff_rank'] = top_coeffs['coeff'].rank(ascending=False).astype(int)
top_coeffs

In [ ]:
print(top_index)
small.loc[top_index].head()

In [ ]:
top_target = small.loc[top_index]['target']
top_target

In [ ]:
cramfile =  cramer_df = '/MFMDatasets/MFM_bopf/data/csl/Cramer-corr-' + top_target + '.csv'
print(cramfile)
cramer_df = pd.read_csv(cramfile, header=None, index_col=0, names = ['Variable', 'Cramer_corr'])
cramer_df['Cramer_rank'] = cramer_df['Cramer_corr'].rank(ascending=False).astype(int)
cramer_df

In [ ]:
compare_df = pd.merge(top_coeffs, cramer_df, how='inner', on='Variable')
compare_df

In [ ]:
#
# @author: Ritesh Agrawal
# @Date: 13 Feb 2013
# @Description: This is an implementation of rank biased overlap score 
# (Refererence: http://www.umiacs.umd.edu/~wew/papers/wmz10_tois.pdf). 
# This is a modified implementation of  https://github.com/maslinych/linis-scripts/blob/master/rbo_calc.py
# It is a linear implementation of the RBO and assumes there are no
# duplicates and doesn't handle for ties. 
#

def score(l1, l2, p = 0.98):
    """
        Calculates Ranked Biased Overlap (RBO) score. 
        l1 -- Ranked List 1
        l2 -- Ranked List 2
    """
    if l1 == None: l1 = []
    if l2 == None: l2 = []
    
    sl,ll = sorted([(len(l1), l1),(len(l2),l2)])
    s, S = sl
    l, L = ll
    if s == 0: return 0

    # Calculate the overlaps at ranks 1 through l 
    # (the longer of the two lists)
    ss = set([]) # contains elements from the smaller list till depth i
    ls = set([]) # contains elements from the longer list till depth i
    x_d = {0: 0}
    sum1 = 0.0
    for i in range(l):
        x = L[i]
        y = S[i] if i < s else None
        d = i + 1
        
        # if two elements are same then 
        # we don't need to add to either of the set
        if x == y: 
            x_d[d] = x_d[d-1] + 1.0
        # else add items to respective list
        # and calculate overlap
        else: 
            ls.add(x) 
            if y != None: ss.add(y)
            x_d[d] = x_d[d-1] + (1.0 if x in ss else 0.0) + (1.0 if y in ls else 0.0)     
        #calculate average overlap
        sum1 += x_d[d]/d * pow(p, d)
        
    sum2 = 0.0
    for i in range(l-s):
        d = s+i+1
        sum2 += x_d[d]*(d-s)/(d*s)*pow(p,d)

    sum3 = ((x_d[l]-x_d[s])/l+x_d[s]/s)*pow(p,l)

    # Equation 32
    rbo_ext = (1-p)/p*(sum1+sum2)+sum3
    return rbo_ext
    

In [ ]:
from scipy.stats import kendalltau, mannwhitneyu
# Calculate Rank-Based Overlap score
rbo = score(compare_df['Cramer_rank'].tolist(), compare_df['coeff_rank'].tolist())
print(f'rbo = {rbo}')
ken_tau = kendalltau(compare_df['Cramer_rank'].tolist(), compare_df['coeff_rank'].tolist())
print(f'ken_tau = {ken_tau}')
mann_whit = mannwhitneyu(compare_df['Cramer_rank'].tolist(), compare_df['coeff_rank'].tolist())
print(f'mann_whit = {mann_whit}')
print(f'CLF_time(min) = {combo.loc[top_index]["CLF_time(min)"]}')
print(f'{SORT_STAT} = {combo.loc[top_index][SORT_STAT]}')


In [ ]:
file = outpath + '/cramer_compare/' + alg + '-' + SORT_STAT + '-CramerCoeffComp.csv'
with open(file, 'w', newline='') as outfile:
    print(f'Algorithm, {alg}', file=outfile)
    print(f'Target, {top_target}', file=outfile)
    print(f'RBO, {rbo:.3f}', file=outfile)
    print(f'Kendall_tau, {ken_tau.correlation:.3f}', file=outfile)
    print(f'CLF_time(min) = {combo.loc[top_index]["CLF_time(min)"]}', file=outfile)
    print(f'{SORT_STAT} = {combo.loc[top_index][SORT_STAT]}\n', file=outfile)
#    print(f'MannWhitneyU, {mann_whit.statistic:.3f}\n', file=outfile)
    
if out_flag:
    compare_df.to_csv(file, mode='a', float_format='%0.6f')

In [ ]:
a1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25]
a2 = [25,24,23,22,21,20,19,18,17,16,15,14,13,12,11,10,9,8,7,6,5,4,3,2,1]
a3 = [1,3,2,4,6,5]
a4 = [5,3,1,4,6,2]
a5 = [24,22,20,18,7,6,4,3,2,1]
print(f'rbo(a1, a1) = {score(a1,a1)}')
print(f'rbo(a1, a2) = {score(a1,a2)}')
print(f'rbo(a1, a3) = {score(a1,a3)}')
print(f'rbo(a1, a4) = {score(a1,a4)}')
print(f'rbo(a1, a5) = {score(a1,a5)}')

In [ ]:
from scipy.stats import kendalltau
print(f'ken_tau(a1, a1) = {kendalltau(a1,a1)}')
print(f'ken_tau(a1, a2) = {kendalltau(a1,a2)}')
print(f'ken_tau(a3, a4) = {kendalltau(a3,a4)}')
#print(f'ken_tau(a1, a4) = {kendalltau(a1,a4)}')
#print(f'ken_tau(a1, a5) = {kendalltau(a1,a5)}')

In [ ]:
print(f'mann_whit(a1, a1) = {mannwhitneyu(a1,a1)}')
print(f'mann_whit(a1, a2) = {mannwhitneyu(a1,a2)}')
print(f'mann_whit(a3, a4) = {mannwhitneyu(a3,a4)}')
#print(f'ken_tau(a1, a4) = {kendalltau(a1,a4)}')
#print(f'ken_tau(a1, a5) = {kendalltau(a1,a5)}')